# Using the Commodity Futures and Options data-set

The R version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-R/CommodityFuturesAndOptions.ipynb)

In [1]:
import pandas as pd    
from sqlalchemy import func, and_, or_, text
from sqlalchemy.orm import aliased
from plutoPy.model import RoutingSession, CommodityFuturesAndOptions
from datetime import date, datetime

pd.set_option('display.max_columns', 500)

### get all the commodities that are being traded in COMEX and NYMEX

In [2]:
end_dt = RoutingSession.session.query(func.max(CommodityFuturesAndOptions.CmeEod.TIME_STAMP)).scalar() 

alias1 = aliased(CommodityFuturesAndOptions.CmeEod)
alias2 = aliased(CommodityFuturesAndOptions.CmeEod)

t1 = (RoutingSession.session.query(alias1.PRODUCT_SYMBOL, alias1.PRODUCT_DESCRIPTION, func.sum(alias1.VOLUME).label("total_volume"))
      .filter(alias1.TIME_STAMP == end_dt)
      .group_by(alias1.PRODUCT_SYMBOL, alias1.PRODUCT_DESCRIPTION)
      .order_by(text("total_volume desc"))
      .all())

t2 = (RoutingSession.session.query(alias1.PRODUCT_SYMBOL, func.min(alias1.TIME_STAMP).label("start_dt"))
      .group_by(alias1.PRODUCT_SYMBOL)
      .order_by(text("start_dt"))
      .all())

pd1 = pd.DataFrame(t1, columns=['SYMBOL', 'DESCRIPTION', 'VOLUME'])
pd2 = pd.DataFrame(t2, columns=['SYMBOL', 'LISTED_DATE'])
tradedContracts = pd.merge(pd1, pd2, on='SYMBOL')

print(tradedContracts[tradedContracts['VOLUME'] > 0])

/usr/local/lib/python3.6/dist-packages/sqlalchemy/dialects/mssql/base.py:2307: SAWarning: Could not fetch transaction isolation level, tried views: ('sys.dm_exec_sessions', 'sys.dm_pdw_nodes_exec_sessions'); final error was: ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'sys.dm_pdw_nodes_exec_sessions'.")
  "tried views: %s; final error was: %s" % (views, last_error)


    SYMBOL                                        DESCRIPTION     VOLUME  \
0       CL                      Light Sweet Crude Oil Futures  1098386.0   
1       GC                                       Gold Futures   565590.0   
2       NG                      Henry Hub Natural Gas Futures   328816.0   
3       SI                                     Silver Futures   182041.0   
4       HG                                     Copper Futures   160775.0   
5       RB                              Rbob Gasoline Futures   155504.0   
6       HO                             Ny Harbor Ulsd Futures   142055.0   
7       BZ         Brent Crude Oil Last Day Financial Futures    99092.0   
8      CLT                              Crude Oil Tas Futures    50149.0   
9      MGC                               E-micro Gold Futures    39972.0   
10      QM                           E-mini Crude Oil Futures    33666.0   
11      PL                                   Platinum Futures    23393.0   
12     RBT  

### get all the commodity futures that are being traded in MCX

In [3]:
end_dt = RoutingSession.session.query(func.max(CommodityFuturesAndOptions.McxEod.TIME_STAMP)).scalar() 

alias1 = aliased(CommodityFuturesAndOptions.McxEod)
alias2 = aliased(CommodityFuturesAndOptions.McxEod)

t1 = (RoutingSession.session.query(alias1.CONTRACT, func.sum(alias1.OI).label("total_oi"))
      .filter(and_(alias1.TIME_STAMP == end_dt, 
                   or_(alias1.OTYPE == 'XX', alias1.OTYPE == 'FUTCOM')))
      .group_by(alias1.CONTRACT)
      .order_by(text("total_oi desc"))
      .all())

t2 = (RoutingSession.session.query(alias1.CONTRACT, func.min(alias1.TIME_STAMP).label("start_dt"))
      .group_by(alias1.CONTRACT)
      .filter(or_(alias1.OTYPE == 'XX', alias1.OTYPE == 'FUTCOM'))
      .order_by(text("start_dt"))
      .all())

pd1 = pd.DataFrame(t1, columns=['SYMBOL', 'OI'])
pd2 = pd.DataFrame(t2, columns=['SYMBOL', 'LISTED_DATE'])
tradedContracts = pd.merge(pd1, pd2, on='SYMBOL')

print(tradedContracts[tradedContracts['OI'] > 0])

        SYMBOL       OI LISTED_DATE
0    CRUDEOILM  43780.0  2015-01-06
1    SILVERMIC  36278.0  2011-02-18
2         GOLD  30740.0  2003-11-10
3        GOLDM  23030.0  2003-11-20
4       SILVER  18557.0  2003-11-10
5   NATURALGAS  17641.0  2006-07-11
6    GOLDPETAL  16666.0  2011-04-18
7       NICKEL  15391.0  2004-06-03
8     CRUDEOIL  14686.0  2005-02-09
9      SILVERM  14598.0  2004-02-17
10      COTTON  10305.0  2011-10-03
11         CPO   7898.0  2004-05-17
12    ZINCMINI   5927.0  2010-05-24
13      COPPER   5519.0  2004-06-04
14     ALUMINI   4166.0  2011-02-28
15        ZINC   3809.0  2006-03-27
16   ALUMINIUM   2456.0  2005-10-26
17    LEADMINI   2137.0  2010-06-01
18        LEAD   1905.0  2006-03-27
19   MENTHAOIL   1491.0  2005-04-26
20  GOLDGUINEA   1343.0  2008-05-08
21    CARDAMOM    265.0  2006-02-14


### get all the commodity futures that are being traded in MCX

In [4]:
end_dt = RoutingSession.session.query(func.max(CommodityFuturesAndOptions.NcdexEod.TIME_STAMP)).scalar() 

alias1 = aliased(CommodityFuturesAndOptions.NcdexEod)
alias2 = aliased(CommodityFuturesAndOptions.NcdexEod)

t1 = (RoutingSession.session.query(alias1.COMMODITY, func.sum(alias1.OI).label("total_oi"))
      .filter(alias1.TIME_STAMP == end_dt)
      .group_by(alias1.COMMODITY)
      .order_by(text("total_oi desc"))
      .all())

t2 = (RoutingSession.session.query(alias1.COMMODITY, func.min(alias1.TIME_STAMP).label("start_dt"))
      .group_by(alias1.COMMODITY)
      .order_by(text("start_dt"))
      .all())

pd1 = pd.DataFrame(t1, columns=['COMMODITY', 'OI'])
pd2 = pd.DataFrame(t2, columns=['COMMODITY', 'LISTED_DATE'])
tradedContracts = pd.merge(pd1, pd2, on='COMMODITY')

print(tradedContracts[tradedContracts['OI'] > 0])


                 COMMODITY        OI LISTED_DATE
0            CASTORSEEDNEW  246595.0  2009-09-03
1              GUAR_SEED10  120550.0  2013-09-05
2       RAPE_MUSTARD_SEEDS   99820.0  2010-11-10
3   COTTONSEEDOILCAKEAKOLA   97780.0  2009-09-03
4              GUARGUM_5MT   66805.0  2016-05-02
5                  SOY_OIL   63830.0  2015-06-01
6                CORIANDER   33780.0  2008-08-11
7                WHEATKOTA   10780.0  2017-12-01
8                MAIZERABI    3300.0  2016-02-26
9                 KAPASNEW    2087.0  2018-06-14
10                  BARLEY    1800.0  2006-12-11
11                   MOONG     735.0  2019-07-08
12           PADDY_BASMATI      80.0  2019-07-10


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)